# Task with graphs

## Helping function and imports

In [14]:
from itertools import combinations, groupby
import random
import networkx as nx
import matplotlib.pyplot as plt


# You can use this function to generate a random graph with 'num_of_nodes' nodes
# and 'completeness' probability of an edge between any two nodes
# If 'directed' is True, the graph will be directed
# If 'draw' is True, the graph will be drawn
def gnp_random_connected_graph(num_of_nodes: int,
                               completeness: int,
                               directed: bool = False,
                               draw: bool = False):
    """
    Generates a random graph, similarly to an Erdős-Rényi
    graph, but enforcing that the resulting graph is conneted (in case of undirected graphs)
    """

    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    edges = combinations(range(num_of_nodes), 2)
    G.add_nodes_from(range(num_of_nodes))

    for _, node_edges in groupby(edges, key=lambda x: x[0]):
        node_edges = list(node_edges)
        random_edge = random.choice(node_edges)
        if random.random() < 0.5:
            random_edge = random_edge[::-1]
        G.add_edge(*random_edge)
        for e in node_edges:
            if random.random() < completeness:
                G.add_edge(*e)

    for (u, v, w) in G.edges(data=True):
        w['weight'] = random.randint(0, 20)

    if draw:
        plt.figure(figsize=(10, 6))
        if directed:
            # draw with edge weights
            pos = nx.arf_layout(G)
            nx.draw(G, pos, node_color='lightblue',
                    with_labels=True,
                    node_size=500,
                    arrowsize=20,
                    arrows=True)
            labels = nx.get_edge_attributes(G, 'weight')
            nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

        else:
            nx.draw(G, node_color='lightblue',
                    with_labels=True,
                    node_size=500)

    return G

## Task 1.1.1. Prim algorithm

In [3]:
from heapq import heappush, heappop

def prim_algo(graph: nx.Graph, start=0):
    """This function applies Prim's algorithm to the given graph.

    Args:
        graph (nx.Graph): original graph
        start (int, optional): node where to start. Defaults to 0.

    Returns:
        nx.Graph: minimum spanning edges
    """
    visited = {start}
    tree = nx.Graph()

    # heapq -- data structure that always looks like SORTED list
    # and the first item is always the least
    heapq = []

    # initialize. add data to heapq
    for node, weight in graph.adj[start].items():
        weight = weight['weight']
        heappush(heapq, (weight, start, node))

    # while there's edges in heapq or not all nodes are in tree
    while heapq and len(visited) != graph.number_of_nodes():
        # get least edge
        weight, node1, node2 = heappop(heapq)

        # this node is already in tree
        if node2 in visited:
            continue

        tree.add_edge(node1, node2, weight=weight)
        visited.add(node2)

        # add all edges from node to N to heapq
        for node, weight in graph.adj[node2].items():
            if node in visited:
                continue

            weight = weight['weight']
            heappush(heapq, (weight, start, node))
        
    return tree

## Task 1.1.2. Kruskal algorithm

In [4]:
def kruskal_algo(graph: nx.Graph) -> nx.Graph:
    """make minimum spanning edges by Kruskal's algorithm

    Args:
        graph (nx.Graph): original graph

    Returns:
        nx.Graph: minimum spanning edges
    """
    trees = [set([i]) for i in graph.nodes()]

    # graph represented in (v1, v2, {'weight': w}) and sorted by weight
    graph = sorted(graph.edges(data=True), key=lambda x: x[2]['weight'])

    result = nx.Graph()

    while graph and len(trees) > 1:
        node1, node2, weight = graph.pop(0)
        weight = weight['weight']

        first_tree, second_tree = None, None

        # find in which trees the first and second nodes are
        for tree in trees:
            if node1 in tree:
                first_tree = tree

            if node2 in tree:
                second_tree = tree

            # found 1 and 2 trees
            if first_tree and second_tree:
                break

        # they are in the same trees,
        # so they would do a cycle if we connect them
        if first_tree == second_tree:
            continue

        # add to result
        result.add_edge(node1, node2, weight=weight)

        # extend first tree with the second by reference
        # (it will change anywhere)
        # and delete second tree
        first_tree.update(second_tree)
        del trees[trees.index(second_tree)]

    return result


## Comparison

In [8]:
from time import time
from networkx.algorithms import tree


for size in [5, 10, 20, 50, 100, 500]:
    graph = gnp_random_connected_graph(size, 0.8)

    start = time()
    prim_algo(graph)
    print(f"Own Prim algorithm with {size} nodes: {time() - start}s")

    start = time()
    tree.minimum_spanning_edges(graph, algorithm='prim')
    print(f"Networkx Prim algorithm with {size} nodes: {time() - start}s")
    print()
    
    start = time()
    kruskal_algo(graph)
    print(f"Own Kruskal algorithm with {size} nodes: {time() - start}s")

    start = time()
    tree.minimum_spanning_edges(graph, algorithm='kruskal')
    print(f"Networkx Kruskal algorithm with {size} nodes: {time() - start}s")
    print()

Own Prim algorithm with 5 nodes: 2.5987625122070312e-05s
Networkx Prim algorithm with 5 nodes: 6.9141387939453125e-06s

Own Kruskal algorithm with 5 nodes: 3.0994415283203125e-05s
Networkx Kruskal algorithm with 5 nodes: 4.76837158203125e-06s

Own Prim algorithm with 10 nodes: 4.00543212890625e-05s
Networkx Prim algorithm with 10 nodes: 2.1457672119140625e-06s

Own Kruskal algorithm with 10 nodes: 4.887580871582031e-05s
Networkx Kruskal algorithm with 10 nodes: 2.1457672119140625e-06s

Own Prim algorithm with 20 nodes: 0.00011491775512695312s
Networkx Prim algorithm with 20 nodes: 9.5367431640625e-07s

Own Kruskal algorithm with 20 nodes: 0.0001289844512939453s
Networkx Kruskal algorithm with 20 nodes: 1.1920928955078125e-06s

Own Prim algorithm with 50 nodes: 0.000997781753540039s
Networkx Prim algorithm with 50 nodes: 5.9604644775390625e-06s

Own Kruskal algorithm with 50 nodes: 0.00074005126953125s
Networkx Kruskal algorithm with 50 nodes: 4.0531158447265625e-06s

Own Prim algorithm

## Task 1.2.1. Bellman-Ford

In [6]:
from math import inf

def bellman_ford_algo(
        graph: nx.Graph,
        start_vertex=0
    ) -> tuple[dict[int, list[int]], dict[int, int]]:
    """Bellman-Ford algorithm. Find the shortest way from selected node to
    every else

    Args:
        graph (nx.Graph): original graph
        start_vertex (int, optional): selected node. Defaults to 0.

    Returns:
        tuple[dict[int, list[int]], dict[int, int]]: 1st dict is the shortest 
            path to current node. 2nd element is lengths of shortest paths
    """
    nodes = graph.nodes
    edges = graph.edges(data=True)

    # initialize lengths and set start to 0
    lengths = {vertex: inf for vertex in nodes}
    lengths[start_vertex] = 0

    # initialize path
    path = {vertex: [] for vertex in nodes}
    path[start_vertex] = [start_vertex]

    # do n - 1 iterations
    for _ in range(len(nodes) - 1):
        # for every edge
        for node1, node2, weight in edges:
            weight = weight['weight']

            # compare current path and path to node1 + path from node1 to node2
            if lengths[node2] > (new_length := lengths[node1] + weight):
                lengths[node2] = new_length
                path[node2] = path[node1] + [node2]

    # control iteration. If smth changes -- there's negative cycle
    for node1, node2, weight in edges:
        weight = weight['weight']

        if lengths[node2] > lengths[node1] + weight:
            return None # because graph have negative cycle

    return path, lengths

## Bellman-Ford comparison

In [20]:
for size in [5, 10, 20, 50, 100, 500]:
    graph = gnp_random_connected_graph(size, 0.8, True)

    start = time()
    bellman_ford_algo(graph)
    print(f"Own Bellman-Ford algorithm with {size} nodes: {time() - start}s")

    start = time()
    nx.single_source_bellman_ford(graph, 0)
    print(f"Networkx Bellman-Ford algorithm with {size} nodes: {time() - start}s")

    print()

Own Bellman-Ford algorithm with 5 nodes: 3.2901763916015625e-05s
Networkx Bellman-Ford algorithm with 5 nodes: 5.316734313964844e-05s

Own Bellman-Ford algorithm with 10 nodes: 0.0001201629638671875s
Networkx Bellman-Ford algorithm with 10 nodes: 5.3882598876953125e-05s

Own Bellman-Ford algorithm with 20 nodes: 0.0008020401000976562s
Networkx Bellman-Ford algorithm with 20 nodes: 0.000141143798828125s

Own Bellman-Ford algorithm with 50 nodes: 0.009967803955078125s
Networkx Bellman-Ford algorithm with 50 nodes: 0.00043010711669921875s

Own Bellman-Ford algorithm with 100 nodes: 0.05308818817138672s
Networkx Bellman-Ford algorithm with 100 nodes: 0.0008261203765869141s

Own Bellman-Ford algorithm with 500 nodes: 5.903329133987427s
Networkx Bellman-Ford algorithm with 500 nodes: 0.013721942901611328s



## Task 1.2.2. Floyd-Warshall

In [24]:
from math import inf

def floyd_warshall(graph: nx.Graph) -> list[list[int | float]] | None:
    """Floyd-Warshall algorithm

    Args:
        graph (nx.Graph): original graph

    Returns:
        list[list[int | float]] | None: distances
    """
    # create adjacency matrix
    nodes_count = len(graph.nodes)
    # init matrix with default value inf
    matrix = [[inf] * nodes_count for _ in range(nodes_count)]

    # set all values
    for edge in graph.edges(data=True):
        matrix[edge[0]][edge[1]] = edge[2]['weight']

    nodes_range = range(nodes_count)

    # set main diagonal to 0
    for i in nodes_range:
        matrix[i][i] = 0

    for k in nodes_range:
        for i in nodes_range:
            for j in nodes_range:
                matrix[i][j] = min(matrix[i][k] + matrix[k][j], matrix[i][j])

            if matrix[i][i] < 0:
                print("Negative cycle detected")
                return None

    return matrix


In [27]:
for size in [5, 10, 20, 50, 100]:
    graph = gnp_random_connected_graph(size, 0.8, True)

    start = time()
    floyd_warshall(graph)
    print(f"Own Floyd-Warshall algorithm with {size} nodes: {time() - start}s")

    start = time()
    nx.floyd_warshall(graph)
    print(f"Networkx Floyd-Warshall algorithm with {size} nodes: {time() - start}s")

    print()

Own Floyd-Warshall algorithm with 5 nodes: 3.1948089599609375e-05s
Networkx Floyd-Warshall algorithm with 5 nodes: 3.790855407714844e-05s

Own Floyd-Warshall algorithm with 10 nodes: 0.00015091896057128906s
Networkx Floyd-Warshall algorithm with 10 nodes: 0.00013375282287597656s

Own Floyd-Warshall algorithm with 20 nodes: 0.001065969467163086s
Networkx Floyd-Warshall algorithm with 20 nodes: 0.0007622241973876953s

Own Floyd-Warshall algorithm with 50 nodes: 0.014951944351196289s
Networkx Floyd-Warshall algorithm with 50 nodes: 0.008567094802856445s

Own Floyd-Warshall algorithm with 100 nodes: 0.10615181922912598s
Networkx Floyd-Warshall algorithm with 100 nodes: 0.06069302558898926s

